In [1]:
import os
import cv2
import numpy as np
import shutil
import subprocess

def classify_image(image_path, threshold=100):
    """
    이미지 파일 경로를 받아 그레이스케일로 변환 후 평균 밝기를 계산하고,
    임계값보다 낮으면 어두운 이미지(True), 그렇지 않으면 밝은 이미지(False)를 반환합니다.
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None, None
    avg_brightness = np.mean(img)
    return avg_brightness, avg_brightness < threshold

# 폴더 경로 설정 (환경에 맞게 수정)
input_folder = r"input"          # 분류할 이미지가 있는 폴더
bright_folder = r"output-br"     # 밝은 이미지를 저장할 폴더
dark_folder = r"output-dk"       # 어두운 이미지를 저장할 폴더
enhanced_folder = r"output-enh"  # colie를 통해 보정된 이미지가 저장될 폴더

# 출력 폴더가 없으면 생성
os.makedirs(bright_folder, exist_ok=True)
os.makedirs(dark_folder, exist_ok=True)
os.makedirs(enhanced_folder, exist_ok=True)

# 분류 임계값 설정 (필요에 따라 조정)
threshold = 50

# 입력 폴더 내 모든 이미지 파일 분류 및 이동
for filename in os.listdir(input_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        continue

    image_path = os.path.join(input_folder, filename)
    brightness, is_dark = classify_image(image_path, threshold)

    if brightness is None:
        print("이미지를 불러올 수 없습니다:", image_path)
        continue

    if is_dark:
        dest_path = os.path.join(dark_folder, filename)
        shutil.move(image_path, dest_path)
        print(f"{filename}: 어두운 이미지 (평균 밝기: {brightness:.2f}) -> {dark_folder}")
    else:
        dest_path = os.path.join(bright_folder, filename)
        shutil.move(image_path, dest_path)
        print(f"{filename}: 밝은 이미지 (평균 밝기: {brightness:.2f}) -> {bright_folder}")

# 이제 어두운 이미지가 모인 dark_folder를 대상으로 colie.py를 실행하여 밝기 보정을 진행합니다.
# colie.py는 [colie GitHub](https://github.com/ctom2/colie)를 참고한 코드로, 
# --input_folder, --alpha, --beta, --gamma, --delta, --L (및 --output_folder 옵션이 있다면)을 인자로 받는다고 가정합니다.

command = [
    "python", os.path.join("colie", "colie.py"),
    "--input_folder", os.path.abspath(dark_folder),
    "--output_folder", os.path.abspath(enhanced_folder),
    "--alpha", "1",
    "--beta", "20",
    "--gamma", "9",
    "--delta", "5",
    "--L", "0.1"
]

print("어두운 이미지 보정을 위해 colie.py 실행 중...")
result = subprocess.run(command)
if result.returncode == 0:
    print("보정 완료! 보정된 이미지는", os.path.abspath(enhanced_folder), "폴더에 저장되었습니다.")
else:
    print("colie.py 실행 중 오류가 발생했습니다.")

# 보정 후, 필요에 따라 enhanced_folder의 이미지를 out-br 폴더로 이동시키거나,
# 별도로 관리할 수 있습니다.

exam-br001.jpg: 밝은 이미지 (평균 밝기: 82.52) -> output-br
exam-br002.jpg: 밝은 이미지 (평균 밝기: 82.94) -> output-br
exam-br003.jpg: 밝은 이미지 (평균 밝기: 82.97) -> output-br
exam-br004.jpg: 밝은 이미지 (평균 밝기: 83.26) -> output-br
exam-br005.jpg: 밝은 이미지 (평균 밝기: 82.56) -> output-br
exam-br006.jpg: 밝은 이미지 (평균 밝기: 82.61) -> output-br
exam-br007.jpg: 밝은 이미지 (평균 밝기: 82.82) -> output-br
exam-br008.jpg: 밝은 이미지 (평균 밝기: 82.71) -> output-br
exam-br009.jpg: 밝은 이미지 (평균 밝기: 81.49) -> output-br
exam-br010.jpg: 밝은 이미지 (평균 밝기: 82.74) -> output-br
exam-br011.jpg: 밝은 이미지 (평균 밝기: 84.01) -> output-br
exam-br012.jpg: 밝은 이미지 (평균 밝기: 88.71) -> output-br
exam-dk001.jpg: 어두운 이미지 (평균 밝기: 26.86) -> output-dk
exam-dk002.jpg: 어두운 이미지 (평균 밝기: 26.94) -> output-dk
exam-dk003.jpg: 어두운 이미지 (평균 밝기: 27.02) -> output-dk
어두운 이미지 보정을 위해 colie.py 실행 중...
보정 완료! 보정된 이미지는 c:\Users\허한성\Desktop\git\License-Plate-Identification\LLIE\output-enh 폴더에 저장되었습니다.
